In [1]:
# Neural Network

In [2]:
import pandas as pd
import os
import numpy as np
from glob import glob

In [63]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [12]:
#import some sample data
# Load data
#2017 psm data
# Lat/lon: 43.85, -99.5
df1_2018 = pd.read_csv(os.path.join(_Testing_Data_Dir,'psm_south_dakota', 'psm_testing_data2018.csv'))
df1_psm_2017= pd.read_csv(os.path.join(_Testing_Data_Dir, 'psm_south_dakota', 'psm_testing_data2017.csv'))

In [13]:
#Lets try and make a neural network

In [14]:
#Get 2018 Data
_Curr_Dir = os.getcwd()
_Testing_Data_Dir = os.path.join(_Curr_Dir, 'Testing_Data')
print(_Testing_Data_Dir)

/Users/tuomastalvitie/Documents/GRIP/Diffuse:Direct/solarIrradiencePredictor/Testing_Data


In [ ]:
"""
So we have all the above variables....Which ones do we think would work best in a NN?

Perhaps a seasonality variable?
Cloud cover for sure
DHI DNI too
Pressure
Solar Zenith Angle

Lets leave the seasonality variable for last
"""

In [ ]:
#Get training arrays and

In [58]:
def combine_training_arrays(df, input_arry, output_arr):
    ghi  = df['GHI'].values
    cloud_cover = df['Cloud Cover'].values
    hours = df['Hour'].values
    minutes = df['Minute'].values
    solar_zenith = df['Solar Zenith Angle'].values
    pressure = df['Pressure'].values
    dhi = df['DHI'].values
    assert len(pressure)==len(solar_zenith)==len(minutes)==len(hours)==len(cloud_cover)==len(ghi)==8760, "len of input array not 8760"
    ar = np.array([ghi, cloud_cover, hours, minutes, solar_zenith, pressure]).T
    input_arry = np.concatenate((input_arry, ar))
    output_arr = np.concatenate((output_arr, np.array(dhi).T))
    return input_arry, output_arr


In [59]:
input_size = 6
input_arry = np.array([], dtype=np.float64).reshape(0,input_size)
output_arry = np.array([], dtype=np.float64)
print(input_arry)
print(output_arry)
from glob import glob
for path in glob(_Testing_Data_Dir+'/*'):
    for file in glob(path+'/psm_'+'*'):
        df = pd.read_csv(file)
        input_arry, output_arry = combine_training_arrays(df, input_arry, output_arry)

[]
[]
[[0.0000e+00 0.0000e+00 0.0000e+00 3.0000e+01 1.1318e+02 1.0190e+03]
 [0.0000e+00 0.0000e+00 1.0000e+00 3.0000e+01 1.2631e+02 1.0190e+03]
 [0.0000e+00 0.0000e+00 2.0000e+00 3.0000e+01 1.3965e+02 1.0190e+03]
 ...
 [1.9900e+02 1.0000e-01 2.1000e+01 3.0000e+01 7.3700e+01 9.4300e+02]
 [1.2200e+02 6.0000e-02 2.2000e+01 3.0000e+01 7.8720e+01 9.4300e+02]
 [3.6000e+01 2.9000e-01 2.3000e+01 3.0000e+01 8.5460e+01 9.4300e+02]]
(429240, 6)
[ 0.  0.  0. ... 96. 68. 28.]
(429240,)
[  0.   0.   0. ... 199. 122.  36.]


In [60]:
#sanity check, make sure 8760*number of files we look at == 429240
count = 0
for path in glob(_Testing_Data_Dir+'/*'):
    for file in glob(path+'/psm_'+'*'):
        count+=1
print(count*8760)

429240


In [61]:
#Second sanity check


In [66]:
model = keras.Sequential()
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(2, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [65]:
#Compile and train
model.compile(optimizer='adam', 
              loss='mse',
              metrics=['mae'])
model.fit(input_arry, output_arry, epochs=10, batch_size=1)

Epoch 1/10
429240/429240 [==============================] - 405s 943us/step - loss: 13807.2021 - mae: 65.1752
Epoch 2/10
429240/429240 [==============================] - 687s 2ms/step - loss: 13807.0283 - mae: 65.1748
Epoch 3/10
429240/429240 [==============================] - 372s 867us/step - loss: 13806.9580 - mae: 65.1752
Epoch 4/10
429240/429240 [==============================] - 335s 780us/step - loss: 13806.9004 - mae: 65.1732
Epoch 5/10
429240/429240 [==============================] - 364s 847us/step - loss: 13807.0996 - mae: 65.1749
Epoch 6/10
429240/429240 [==============================] - 337s 785us/step - loss: 13806.9922 - mae: 65.1751
Epoch 7/10
429240/429240 [==============================] - 334s 778us/step - loss: 13806.9922 - mae: 65.1742
Epoch 8/10
429240/429240 [==============================] - 346s 807us/step - loss: 13807.0605 - mae: 65.1757
Epoch 9/10
429240/429240 [==============================] - 334s 779us/step - loss: 13807.0010 - mae: 65.1741
Epoch 10/10


In [67]:
#Compile and train
model.compile(optimizer='adam', 
              loss='mse',
              metrics=['mae'])
model.fit(input_arry, output_arry, epochs=10, batch_size=32)

Epoch 1/10
13414/13414 [==============================] - 13s 962us/step - loss: 13805.1934 - mae: 65.3293
Epoch 2/10
13414/13414 [==============================] - 11s 786us/step - loss: 13804.4980 - mae: 65.3272
Epoch 3/10
13414/13414 [==============================] - 12s 875us/step - loss: 13804.4795 - mae: 65.3275
Epoch 4/10
13414/13414 [==============================] - 11s 784us/step - loss: 13804.4746 - mae: 65.3274
Epoch 5/10
13414/13414 [==============================] - 11s 798us/step - loss: 13804.4717 - mae: 65.3273 0s - loss: 13793.324
Epoch 6/10
13414/13414 [==============================] - 11s 784us/step - loss: 13804.4629 - mae: 65.3274
Epoch 7/10
13414/13414 [==============================] - 11s 783us/step - loss: 13804.4619 - mae: 65.3276s - loss: 13795.458 - ETA: 0s - loss: 13808.3164 - mae: 65. - ETA: 0s -
Epoch 8/10
13414/13414 [==============================] - 11s 825us/step - loss: 13804.5146 - mae: 65.3272
Epoch 9/10
13414/13414 [============================

In [68]:
model = keras.Sequential()
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(5, activation="relu"))
model.add(layers.Dense(10, activation="relu"))
model.add(layers.Dense(20, activation="relu"))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(3, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [69]:
#Compile and train
model.compile(optimizer='adam', 
              loss='mse',
              metrics=['mae'])
model.fit(input_arry, output_arry, epochs=10, batch_size=32)

Epoch 1/10
13414/13414 [==============================] - 13s 959us/step - loss: 13804.7383 - mae: 65.3275
Epoch 2/10
13414/13414 [==============================] - 13s 985us/step - loss: 13804.5010 - mae: 65.3274
Epoch 3/10
13414/13414 [==============================] - 13s 936us/step - loss: 13804.4893 - mae: 65.3274
Epoch 4/10
13414/13414 [==============================] - 12s 919us/step - loss: 13804.4834 - mae: 65.3272
Epoch 5/10
13414/13414 [==============================] - 12s 913us/step - loss: 13804.4746 - mae: 65.3274
Epoch 6/10
13414/13414 [==============================] - 13s 940us/step - loss: 13804.5146 - mae: 65.3273- ETA
Epoch 7/10
13414/13414 [==============================] - 13s 957us/step - loss: 13804.4961 - mae: 65.3274
Epoch 8/10
13414/13414 [==============================] - 13s 954us/step - loss: 13804.4590 - mae: 65.3276
Epoch 9/10
13414/13414 [==============================] - 13s 963us/step - loss: 13804.5479 - mae: 65.3273
Epoch 10/10
13414/13414 [=======

In [70]:
model = keras.Sequential()
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(5, activation="relu"))
model.add(layers.Dense(10, activation="relu"))
model.add(layers.Dense(20, activation="relu"))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(3, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [71]:
#Compile and train
model.compile(optimizer='adam', 
              loss='mse',
              metrics=['mae'])
model.fit(input_arry, output_arry, epochs=10, batch_size=16)

Epoch 1/10
26828/26828 [==============================] - 24s 909us/step - loss: 13804.5312 - mae: 65.3274s - loss: 13795.0850 - ma
Epoch 2/10
26828/26828 [==============================] - 23s 848us/step - loss: 13804.4873 - mae: 65.3276 17s - lo - E
Epoch 3/10
26828/26828 [==============================] - 25s 929us/step - loss: 13804.5225 - mae: 65.3272 21s - loss: 13935.7852 - ma - ETA: 21s - ETA: 19s - loss: 13805.4932 - mae: - ETA - ETA: 18s - loss: 13811.4014 - mae - ETA: 18s - loss: 13865. - ETA: 17s - loss: 13834.8486 - - ETA: 16s - loss: 13829.2031 - mae: 6 - ETA: 15s - loss: - ETA: 14s - loss: 13793 - - ETA: 11s - loss: 13836.7490 - mae: 65.432 - ETA: 11s - loss: 13836.5537 - mae: 65 - ETA: 10s - loss: 13827.7812 - mae: 6 - ETA - ETA: 7s - loss: 13837.4785 - mae: 65.43 - ETA: 7s - l - ETA: 6s -
Epoch 4/10
26828/26828 [==============================] - 25s 930us/step - loss: 13804.4805 - mae: 65.3271s - loss: 13801.2383 - mae:
Epoch 5/10
26828/26828 [=========================

In [74]:
model = keras.Sequential()
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(5, activation="relu"))
model.add(layers.Dense(10, activation="relu"))
model.add(layers.Dense(20, activation="relu"))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(3, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [75]:
#Compile and train
model.compile(optimizer='adam', 
              loss='mse',
              metrics=['mae'])
model.fit(input_arry, output_arry, epochs=10, batch_size=1)

Epoch 1/10
429240/429240 [==============================] - 341s 794us/step - loss: 13936.9922 - mae: 65.3186 - los -
Epoch 2/10
429240/429240 [==============================] - 535s 1ms/step - loss: 13937.2207 - mae: 65.3185
Epoch 3/10
429240/429240 [==============================] - 701s 2ms/step - loss: 13936.9053 - mae: 65.3178
Epoch 4/10
429240/429240 [==============================] - 825s 2ms/step - loss: 13937.1094 - mae: 65.31850s - loss: 13937.1445 - mae:
Epoch 5/10
429240/429240 [==============================] - 792s 2ms/step - loss: 13937.1094 - mae: 65.3181
Epoch 6/10
429240/429240 [==============================] - 647s 2ms/step - loss: 13937.1387 - mae: 65.31760s - loss: 13936.0713 - ma
Epoch 7/10
429240/429240 [==============================] - 708s 2ms/step - loss: 13937.1484 - mae: 65.3174
Epoch 8/10
429240/429240 [==============================] - 428s 998us/step - loss: 13937.0996 - mae: 65.3178
Epoch 9/10
429240/429240 [==============================] - 395s 921us